In [ ]:
import sys
import numpy as np

# TensorFlow and tf.keras
import tensorflow.compat.v1 as tf
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras import utils as np_utils 
from keras.preprocessing.text import Tokenizer

In [ ]:
# loading the text file and convert texts to lowercase
with open('/content/drive/MyDrive/data/shakespeare.txt', 'r') as f:
    text = f.read().lower()

# create a set of all unique characters in the text
chars = sorted(list(set(text)))

In [ ]:
# create a dictionary to map characters to integers and vice versa
char_to_int = dict((c, i) for i, c in enumerate(chars))
idx_to_char = {i: c for i, c in enumerate(chars)}

In [ ]:
# size of the vocabulary

vocab_size = len(chars)
vocab_size

38

In [ ]:
# Number of characters

n_chars = len(text)
n_chars

93677

In [ ]:
# prepare the dataset of input to output encoded as integers 

seq_length = 40
X = []
Y = []
for i in range(0, n_chars-seq_length,3):
 seq_in = text[i:i + seq_length]
 seq_out = text[i + seq_length]
 X.append([char_to_int[char] for char in seq_in])
 Y.append(char_to_int[seq_out])

In [ ]:
n_patterns = len(X)
print ("Total Patterns: "), n_patterns

Total Patterns: 


(None, 93657)

In [ ]:
X = np.reshape(X, (n_patterns, seq_length, 1))

X = X / float(vocab_size)   # normalize
y = to_categorical(Y)   # one hot encode the output variable

In [ ]:
import tensorflow.keras as keras

# define the LSTM model

model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [ ]:
# define the checkpoint
filepath="/content/drive/MyDrive/Weights/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer="adam") #optimize the model

In [ ]:
# load the network weights

filename = "/content/drive/MyDrive/data/weights-improvement-914-0.0221.hdf5"
model.load_weights(filename, True)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 512)               1052672   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 38)                19494     
                                                                 
Total params: 1,072,166
Trainable params: 1,072,166
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for layer in model.layers: print(layer.get_config(), layer.get_weights())       # display the weights which were loaded

{'name': 'lstm', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, 20, 1), 'return_sequences': False, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'time_major': False, 'units': 512, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'unit_forget_bias': True, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.0, 'recurrent_dropout': 0.0, 'implementation': 2} [array([[-0.35341662,  0.44983524, -0.9613208 , ...,  0.08878878,
         0.2943596 , -0.6850567 ]], dtype=float32), array([[ 1.6965122 , -0.35287204, -0.74266505, ...,

### Predicting and Sampling : Generate 100-character sequence.

In [ ]:
# generate characters (1000)

def generate_text(text):
    text = str (text)
    vector=[char_to_int[char] for char in text [-seq_length:]]
    output = text
    for i in range(1000):
        iseq = np.reshape(vector,(1,len(vector), 1))
        iseq = iseq / float(len(chars))

        pred = np.argmax(model.predict(iseq, verbose=0))
        seq = [idx_to_char[value] for value in vector]

        output += idx_to_char[pred]

        vector.append(pred)
        vector = vector[1:len(vector)]

    return output

In [ ]:
#Input sequences for testing

shapespeare1 = "Look in thy glass and tell the"
shapespeare2 = "Thy youth's proud livery so gazed on now,"
shapespeare3 = "leads summer on"

In [ ]:
#Text generation 1

output1 = generate_text(shapespeare1.lower())
print(output1)

look in thy glass and tell the face thou viewest,
now is the time that face should form another,
whose fresh repair if now thou not renewest,
thou dost beguile the world, unbless some mother.
for where is she so fair whose uneared womb
disdains the tillage of thy husbandry?
or who is he so fond will be the tomb,
of his self-love to stop posterity?  
thou art thy mother's glass and she in thee
calls back the lovely april of her prime,
so thou through windows of thine age shalt see,
despite of wrinkles this thy golden time.
but if thou live remembered not to be,
die single and thine image dies with thee.

unthrifty loveliness why dost thou spend,
upon thy self thy beauty's legacy?
nature's bequest gives nothing but doth lend,
and being frank she lends to those are free:
then beauteous niggard why dost thou abuse,
the bounteous largess given thee to give?
profitless usurer why dost thou use
so great a sum of sums yet canst not live?
for having traffic with thy self alone,
thou of thy self

### 4. Evaluation : Generate a 400-character sequence

In [ ]:
# generate characters (400)

def generate_text_400(text):
    text = str (text)
    vector=[char_to_int[char] for char in text [-seq_length:]]
    output = text
    for i in range(400):
        iseq = np.reshape(vector,(1,len(vector), 1))
        iseq = iseq / float(len(chars))

        pred = np.argmax(model.predict(iseq, verbose=0))
        seq = [idx_to_char[value] for value in vector]

        output += idx_to_char[pred]

        vector.append(pred)
        vector = vector[1:len(vector)]

    return output

In [ ]:
#Text generation 2

shakespeare_text1 = generate_text_400(shapespeare1.lower())
print(shakespeare_text1)

look in thy glass and tell the face thou viewest,
now is the time that face should form another,
whose fresh repair if now thou not renewest,
thou dost beguile the world, unbless some mother.
for where is she so fair whose uneared womb
disdains the tillage of thy husbandry?
or who is he so fond will be the tomb,
of his self-love to stop posterity?  
thou art thy mother's glass and she in thee
calls back the lovely april of her


In [ ]:
#Text generation 2

shakespeare_text2 = generate_text_400(shapespeare2.lower())
print(shakespeare_text2)

thy youth's proud livery so gazed on now,
will be a tattered weed of small worth held:  
then being asked, where all thy beauty lies,
where all the treasure of thy lusty days;
to say within thine own deep sunken eyes,
were an all-eating shame, and thriftless praise.
how much more praise deserved thy beauty's use,
if thou couldst answer 'this fair child of mine
shall sum my count, and make my old excuse'
proving his beauty by succession t


In [ ]:
#Text generation 3

shakespeare_text3 = generate_text_400(shapespeare3.lower())
print(shakespeare_text3)

leads summer on thee,
the parn mf toue that lo toue sote canken th thee,
whene thou mhs'st tho whrl woite no wamde tho benuty oafn the blned that benngd the clne so thee,
when yhu bet ey lans ball ant fere whel thou dest ro geed the cank shee thet goeee
th thee,
where yhu hen lo heart which tho geld she blnude to the benean whth thee wout pelf io whnter stblt,
what shou dost banu wilh then ganeen flr thy love, t
